In [ ]:
import pandas as pd
import getpass
from sqlalchemy import create_engine
from sqlalchemy.exc import ResourceClosedError
import MySQLdb
check = False

In [ ]:
class MySQLConnection:
    def __init__(self,user,hostname,db):
        self.user = str(user)
        self.hostname = str(hostname)
        self.db = str(db)
        pword = getpass.getpass("Enter password for user {}".format(user))
        # self.cnx = mysql.connector.connect(host=self.hostname,user=self.user, password=pword,database=self.db)
        self.engine = create_engine("mysql+mysqlconnector://{}:{}@{}/{}".format(user,pword,hostname,db))
        self.cnx = self.engine.connect()
    
    def write_to_db(self, df, table_name):
        try:
            df.to_sql(table_name,con = self.cnx,if_exists='fail',index=False)
        except Exception as e:
            print("\n SQL Write error with: ")
            print(df,"\n", e)
    
    def query(self,q):
        try:
            res = pd.read_sql(q,self.cnx)
            return res
        except ResourceClosedError:
            print("Query success, nothing was returned")

In [ ]:
sql = MySQLConnection('ubuntu','localhost','dati')

In [ ]:
a_code = pd.read_csv('access-code.csv',';')
a_code_pword_rec = pd.read_csv('access-code-password-recovery-code.csv',sep=';')
un_pw_rec_code = pd.read_csv('username-password-recovery-code.csv',sep=';')
un = pd.read_csv('username.csv',sep=';')
email = pd.read_csv('email.csv',sep=';')
un_email = pd.read_csv('username-or-email.csv',sep=';')
email_pw_rec = pd.read_csv('email-password-recovery-code.csv',sep=';')

In [ ]:
a_code.columns = ['Identifier', 'Firstname','Lastname']
a_code_pword_rec.columns = ['Identifier','Accesscode','Recoverycode','Firstname','Lastname','Department','Location']
un_pw_rec_code.columns = ['Username','Identifier','Onetimepassword','Recoverycode','Firstname','Lastname','Department','Location']
un.columns = ['Username', 'Identifier', 'Firstname','Lastname']
email.columns = ['Loginemail','Identifier', 'Firstname','Lastname']
un_email.columns = ['Username','Loginemail','Identifier', 'Firstname','Lastname']
email_pw_rec.columns = ['Loginemail','Identifier','Onetimepassword','Recoverycode', 'Firstname','Lastname','Department','Location']

In [ ]:
# # ŠĪ ŠŪNA JĀIZPILDA VIENREIZ!

# if not check:
#     sql.write_to_db(a_code,'access_code')
#     sql.write_to_db(a_code_pword_rec,'access_code_password_recovery_code')
#     sql.write_to_db(un_pw_rec_code,'username_password_recovery_code')
#     sql.write_to_db(un,'username')
#     sql.write_to_db(email,'email')
#     sql.write_to_db(un_email,'username_or_email')
#     sql.write_to_db(email_pw_rec,'email_password_recovery_code')
#     check = True

# INSERT INTO    
INSERT INTO komnada ir paredzēta rindiņu ierakstīšanai tabulā. Insert ir divi paveidi.  
INSERT INTO \< tabulas nosauk \> VALUES ( \< vērtības sarakstītas atbilstoši kolonnu secībai \> )  
INSERT INTO \< tabulas nosauk \> \< select apgalvojums, kurš atgriež kolonnas secībai, kas atbilsti tabulai, kurā tiek veikts ieraksts \>

In [ ]:
# Insert into values piemērs:
sql.query("select * from email") #Paskatamies, kas atrodās tabulā email

In [ ]:
# Pirms kaut ko ievadam tabulā, izveidojam tabulas email dublējumu

# Uzdevums: Izveidot tabulu email_cp, kura ir identiska tabulai email
sql.query()

In [ ]:
# Pirms ejam tālāk, pārliecinamies, ka tabula email tiešām ir duplicēta
sql.query("select * from email_cp")

In [ ]:
insert_q = """
insert into table email values(vards.uzvards@example.com, 5678, vards, uzvards)
"""

# Vai šis nostrādās? Ko mēs sagaidītu rezultātā?

# sql.query(insert_q)

In [ ]:
# Insert into select piemērs:
# Pieņemsim, ka vēlamies papildināt email tabulu ar ierakstiem no tabulas username_or_email. No sākuma to apskatīsim.
sql.query("select * from username_or_email")

In [ ]:
insert_select_q = """ 
insert into email 
select 
    loginemail,
    identifier, 
    firstname, 
    lastname 
from  
    username_or_email
where 
    loginemail is not NULL
"""

# Divas lietas, ko šeit ievērot:

# 1. Šeit tikai lietots WHERE filtrs. Vai ir kāds arguments, lai to nelietotu šajā gadījumā? Un vispārīgākā?

# 2. Lai gan pandas DataFrame mums trūkstošās vērtības apraksta kā 'None', SQL pareizā sintakse ir NULL
# tāpēc 


In [ ]:
sql.query(insert_select_q)

In [ ]:
# Apskatīsim kā tabula email izskatās tagad
sql.query("select * from email")

In [ ]:
# Mums tabulā ir izveidojušās duplikātrindas. Vienam indeksam vairs neatbilst tikai viens ieraksts. 
# Šo situāciju var labot sekojoši:

sql.query("create table email_table as select distinct * from email") # Uztaisam tabulu tikai ar unikālajiem ierakstiem no tabulas email
sql.query("drop table email") # Izdzēšam email tabulu
sql.query("rename table email_table to email") # pārsaucam unikālo email iekrastu tabulu par email

# DELETE, ALTER TABLE un DROP funkcijas  
DELETE vispārīgā gadījumā ir paredzēts, lai bāzējoties uz kritēriju dzēstu rindiņas izvēlētā tabulā.  
DROP vispārīgā gadījumā tiek lietots, lai izdzēstu izvēlēto tabulu, kolonnu vai datubāzi  


In [ ]:
# Ievietosim tabulā username jaunu lietotāju, ko pēc tam izdzēsīsim:
sql.query("insert into username values('arthur90', 3971, 'Arthur','Cohen')")

In [ ]:
sql.query("select * from username") # Apskatīsim username tabulu

In [ ]:
# Tagad izdzēsīsim tabulas username ierakstu
sql.query("delete from username where Identifier=3971")

In [ ]:
# Apskatīsimies vai ieraksts tiešām ir pazudis:
sql.query("select * from username")

In [ ]:
# Uzdevums:
# Craig Johnson ir piešķirts jauns lietotājvārds. skat. nākamo rindiņu.
sql.query("insert into username values('cjohnson', 4081, 'Craig','Johnson')")
# Izdzēsiet vecā lietotājvārda ierakstu

In [ ]:
# Jautājums:
# Vai delete where būtu lietojams, lai atbrīvotos no duplikātiem?

DELETE tika lietots, lai dzēstu rindas, bet neietekmēja datubāzes shēmu (datubāzu, tabulu un kolonnu nosaukumus un tipus). Tipiski visa informācija tabulās tiek glabāta nevis kolonnās, bet rindās. Rindas ir dinamiskas, bet shēma paliek nemainīga (statiska).  

Tomēr, lai gan shēmu nav paredzēts dinamiski mainīt, to ir iespējams darīt ar DROP.
DROP funkcija var dzēst datubāzes, tabulas un tabulu kolonnas.  
DROP funkciju pirms tam esam lietojuši, lai dzēstu datubāzes un tabulas, bet to var lietot, lai dzēstu arī kolonnas.

In [ ]:
# Uzdevums:
# 1.Izveidojiet tabulu tests, kurā ir 3 kolonnas: 
# a - integer tipa kolonna, b - text tipa kolonna, c - mainīgu simbolu (20) virkne

# 2. Ievietojiet šajā tabulā trīs rindiņas ar patvaļīgiem datiem.

Ja mēs gribētu no šīs tabulas izdzēst kolonnu, to varētu darīt līdzīgi, kā likvidējām duplikātus.  
1. Izveidojam tabulas kopiju, kurā nav nevēlamās kolonnas
2. DROP sākotnējo tabulu
3. Pārsaucam kopiju sākotnējās tabulas vārdā

Savādāk, varam izmantot DROP komandu, ievietojot to zem ALTER TABLE.  

In [ ]:
sql.query("alter table tests drop column a")

In [ ]:
sql.query("select * from tests") #Tagad apskatīsim kā tabula izskatās tagad

ALTER TABLE lieto, lai arī pievienotu kolonnas. Vispārīgā sintake ir sekojoša:  
ALTER TABLE \< tabulas nosaukums \> ADD COLUMN \< kolonnas nosaukums \> \< kolonnas tips \>

In [ ]:
# Kolonnas pievienošanas piemērs:
sql.query("alter table tests add column teksta_kolonna text")

# UPDATE funkcija  
UPDATE strādā ļoti līdzīgi kā DELETE. Rindās, kuras atbilst kritērijam, norādīto kolonnu vērtības tiek nomainītas uz citām.

In [ ]:
# Veiksim šo darbību ar username_or_email tabulu.
sql.query("select * from username_or_email")

In [ ]:
sql.query("update username_or_email set loginemail = 'Nenoradits' where loginemail is NULL")

# Jautājums: 
# Kāda priekšrocība varētu būt paredzēto NULL vērtību iestādīšanai uz kādu specifisku vērtību?

# Aliasi  
SELECT statementā ir iespējams pārsaukt kolonnas vai tabulas. Praktiski tas bieži tiek lietots JOIN statementos, tabulu vietā lietojot citu SELECT rezultātus un ar COUNT/SUM/AVG/MAX/MIN funkciju rezultātu pārsaukšanu.  
Aliasam piesaistītais atslēgvārds ir AS  

In [ ]:
# Piemērs:
# Mēs vēlamies SELECT outputā pārsaukt kolonnas, 
# lai pēc tam ar JOIN funkciju būtu viegli un pārskatāmi uz tām atsaukties

alias_q_example = """
select 
    Identifier as id,
    location as lc,
    department as dp
from
    username_password_recovery_code
"""

sql.query(alias_q_example)

In [ ]:
# Piemērs:
# Pieņemsim mums priekš kopsavilkuma reportā ir jāsaskaita cik mums ir darbinieku Mančesterā.
sql.query("select count(*) as MN_employee_count from username_password_recovery_code where location = 'Manchester'")

# JOIN funkcijas prakse  
http://www.sql-join.com/sql-join-types

In [ ]:
# No sākuma uztaisīsim tabulas
# Uzdevums: 
# 1. Ielādējiet no sekojošajiem linkiem csv failus un ielieciet tos datubāzē dati kā tabulas customers un orders
# https://raw.githubusercontent.com/reversedego/RCS_course/master/3_sql_DBs/customer.csv
# https://raw.githubusercontent.com/reversedego/RCS_course/master/3_sql_DBs/order.csv


In [ ]:
sql.write_to_db(order,'orders')

In [ ]:
# 2. Salabojiet sekojošo vaicājumu, lai tiktu atgriezta tikai kurjeram svarīgā informācija
query = """
select * from customers join orders on customers.customer_id = orders.customer_id
"""
# sql.query(query)

In [ ]:
# 3. Saskaitiet cik kopā ieņēmumu būs no visiem pasūtījumiem kopā?

In [ ]:
# 4. Katrā pilsētā ir tieši viens kurjers, cik piegāžu būs jāveic katram kurjeram?

In [ ]:
# 5. Saskaitiet cik kopā ieņēmumu būs no visiem pasūtījumiem, kas tiek nogādāti uz Charlottesville pilsētu.

In [ ]:
# 6. Cik kurjeri tiks nodarbināti ar šiem pasūtījumiem?

In [ ]:
# 6. Izveidojiet tabulu deliveries, kurā ir: 
# delivery_id - delivery tabulas primārā atslēga, 
# order_id - atsauce uz orders tabulu, 
# Kurjera vārds, uzvārds un telefona numurs,
# Piegādes statuss - nepiegādāts / ceļā / piegādāts (implementējiet kā vēlaties)

# Jautājums: 
# Kad varētu būt laba ideja neveidot kurjeriem atsevišķu tabulu?

In [ ]:
# 7. Pievienojiet tabulai orders visas datu kolonnas un datus no tabulas deliveries. 


# Jautājums: 
# Šādi apvienot kolonnas šajā gadījumā ir laba vai slikta ideja? Kāpēc?